## The Ground Modeling Challenge for AgriPV Application

<img src="MicrosoftTeams-image.png">

### Solution Steps ###
- Setup of variables
- Generating the scenes
- Mapping the ground irradiance

**Given that this is a 1-HSAT Tracker Routine, the workflow we follow is:**

    - set1axis(gets angles)
    - makeScene1axis
    - gendaylit1axis
    - makeoct1axis
    - analysis1axis

## 1. Load Bifacial Radiance and other essential packages

In [40]:
import bifacial_radiance
import numpy as np
import os # this operative system to do the relative-path testfolder for this example.
import pprint    # We will be pretty-printing the trackerdictionary throughout to show its structure.
from pathlib import Path

## 2. Define all the system variables

In [41]:
testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP')


timestamp = 4020 # Noon, June 17th.
simulationName = 'GroundChallenge'    # Optionally adding a simulation name when defning RadianceObj

# Surface    
#albedo = " green grass", which is not one of the default choices in the material list

#Location
lat = 40.1217  # Given for the project site at Colorado
lon = -105.1310  # Given for the project site at Colorado

# MakeModule Parameters

moduletype='PrismSolar'
#numpanels = 1  # This site have 1 module in Y-direction
x = 1  
y = 2
#xgap = 0.15 # Leaving 15 centimeters between modules on x direction
#ygap = 0.10 # Leaving 10 centimeters between modules on y direction
zgap = 0 # no gap to torquetube.
sensorsy = 6  # this will give 6 sensors per module in y-direction
sensorsx = 3   # this will give 3 sensors per module in x-direction

torquetube = True
axisofrotationTorqueTube = True 
diameter = 0.15  # 15 cm diameter for the torquetube
tubetype = 'square'    # Put the right keyword upon reading the document
material = 'black'   # Torque tube of this material (0% reflectivity)

# Scene variables
nMods = 30
nRows = 7
hub_height = 1.8 # meters
pitch = 5.1816 # meters      # Pitch is the known parameter 
albedo = 0.2  #'Grass'     # ground albedo

#azimuth_ang=180 # Facing south 
#axis_azimuth should have a default value of 180

cumulativesky = False
limit_angle = 60 # tracker rotation limit angle
angledelta = 0.01 # we will be doing hourly simulation, we want the angle to be as close to real tracking as possible.
backtrack = True 

## 3. Create Radiance Object including Albedo and Weather

In [42]:
demo = bifacial_radiance.RadianceObj(simulationName,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) 
metdata = demo.readEPW(epwfile, coerce_year = 2021)

path = C:\Users\sarefeen\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_CO_Boulder-Broomfield-Jefferson.County.AP.724699_TMY3.epw
 ... OK!
Saving file EPWs\epw_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


## 4. Make Module

In [43]:
moduleDict = demo.makeModule(name=moduletype, x = x, y =y, torquetube=torquetube, diameter=diameter, tubetype=tubetype, material=material, 
                zgap=zgap, axisofrotationTorqueTube=axisofrotationTorqueTube)



Module Name: PrismSolar
Module PrismSolar updated in module.json


At this point we could use the gendaylit() function for the single timestamp and generate the oct file as below:

In [ ]:
# Now let's run the example

demo = bifacial_radiance.RadianceObj(simulationName,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) 
epwfile = demo.getEPW(lat, lon) 
metdata = demo.readEPW(epwfile)
demo.gendaylit(timestamp)  # We are simulating only one hour at a time. 

sceneDict = {'pitch': pitch,'hub_height':hub_height,'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 30 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.

## 5. Calculate GCR

In [7]:
cw = 1  # Collector Width, CW = 1 as given
gcr = cw/pitch
print("GCR:",gcr)

GCR: 0.1929905820595955


## 6. Generate the Sky for the Tracking Angles

In [51]:
startdate = '06/17'     
enddate = '06/18' #In this case, we are looking to generate tracking scenarios for one day as opposed to a single hour

trackerdict = demo.set1axis(metdata = metdata, limit_angle = limit_angle, backtrack = backtrack, 
                            gcr = gcr, cumulativesky = cumulativesky)

In [46]:
#checking our trackdict

print ("\nTrimmed trackerdict by gendaylit1axis to start and enddate length: %s " % (len(trackerdict)))
print ("")
trackerkeys = sorted(trackerdict.keys())
print ("Option of hours are: ", trackerkeys)
print ("")
print ("Contents of trackerdict for sample hour:")
pprint.pprint(trackerdict[trackerkeys[5]])


Trimmed trackerdict by gendaylit1axis to start and enddate length: 4381 

Option of hours are:  ['2021_01_01_08', '2021_01_01_09', '2021_01_01_10', '2021_01_01_11', '2021_01_01_12', '2021_01_01_13', '2021_01_01_14', '2021_01_01_15', '2021_01_01_16', '2021_01_02_08', '2021_01_02_09', '2021_01_02_10', '2021_01_02_11', '2021_01_02_12', '2021_01_02_13', '2021_01_02_14', '2021_01_02_15', '2021_01_02_16', '2021_01_03_08', '2021_01_03_09', '2021_01_03_10', '2021_01_03_11', '2021_01_03_12', '2021_01_03_13', '2021_01_03_14', '2021_01_03_15', '2021_01_03_16', '2021_01_04_08', '2021_01_04_09', '2021_01_04_10', '2021_01_04_11', '2021_01_04_12', '2021_01_04_13', '2021_01_04_14', '2021_01_04_15', '2021_01_04_16', '2021_01_05_08', '2021_01_05_09', '2021_01_05_10', '2021_01_05_11', '2021_01_05_12', '2021_01_05_13', '2021_01_05_14', '2021_01_05_15', '2021_01_05_16', '2021_01_06_08', '2021_01_06_09', '2021_01_06_10', '2021_01_06_11', '2021_01_06_12', '2021_01_06_13', '2021_01_06_14', '2021_01_06_15', '

## Make Scene1 Axis

In [49]:
# making the different scenes for the 1-axis tracking for the dates in trackerdict2.

sceneDict = {'pitch': pitch,'hub_height':hub_height, 'nMods':nMods, 'nRows': nRows}  

In [50]:
sceneDict

{'pitch': 5.1816, 'hub_height': 1.8, 'nMods': 30, 'nRows': 7}

## Make the 1-axis Tracking Scene and Analyse Irradiance on Module

In [53]:
scene = demo.makeScene1axis(moduletype=moduletype,sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
trackersdict = demo.gendaylit1axis(startdate=startdate, enddate=enddate)
octfile = demo.makeOct1axis()
analysis = demo.analysis1axis()


Making ~15 .rad files for gendaylit 1-axis workflow (this takes a minute..)
15 Radfiles created in /objects/
Creating ~12 skyfiles.  Takes 1-2 minutes
Created 15 skyfiles in /skies/

Making 15 octfiles in root directory.
Created 1axis_2021_06_17_05.oct
Created 1axis_2021_06_17_06.oct
Created 1axis_2021_06_17_07.oct
Created 1axis_2021_06_17_08.oct
Created 1axis_2021_06_17_09.oct
Created 1axis_2021_06_17_10.oct
Created 1axis_2021_06_17_11.oct
Created 1axis_2021_06_17_12.oct
Created 1axis_2021_06_17_13.oct
Created 1axis_2021_06_17_14.oct
Created 1axis_2021_06_17_15.oct
Created 1axis_2021_06_17_16.oct
Created 1axis_2021_06_17_17.oct
Created 1axis_2021_06_17_18.oct
Created 1axis_2021_06_17_19.oct
Linescan in process: 1axis_2021_06_17_05_Front
Linescan in process: 1axis_2021_06_17_05_Back
Saved: results\irr_1axis_2021_06_17_05.csv
Index: 2021_06_17_05. Wm2Front: 5.909456. Wm2Back: 0.5835093
Linescan in process: 1axis_2021_06_17_06_Front
Linescan in process: 1axis_2021_06_17_06_Back
Saved: r

Analysis of Ground Irradiance

In [57]:
octfile1 = 'Created 1axis_2021_06_17_12.oct'
analysis1 = bifacial_radiance.AnalysisObj(octfile1, demo.name)  # return an analysis object including the scan dimensions for back irradiance
sensorsy = 6
frontscan, backscan = analysis1.moduleAnalysis(scene, sensorsy=sensorsy)

AttributeError: 'dict' object has no attribute 'sceneDict'

In [ ]:
groundscan = frontscan

In [ ]:
# This is where we are going to tweak

groundscan['zstart'] = 0.01  # setting it 0.01 cm from the ground simulating for ground surface irradiation
groundscan['zinc'] = 0   # no tilt necessary. 
groundscan['yinc'] = pitch/(sensorsy-1)   # no tilt necessary. 
groundscan

In [ ]:
analysis.analysis(octfile, simulationName+"_groundscan", groundscan, backscan)  # compare the back vs front irradiance

In [ ]:
analysis.Wm2Front

In [ ]:
demo.makeOct1axis(singleindex='06_17_12')
results = demo.analysis1axis(singleindex='06_17_12')
print('\n\nHourly bifi gain: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))

### Code-looping for all hours of the day

In [ ]:
# If we are showing the result for the single hour
print ("\n Contents of trackerdict for sample hour after analysis1axis: ")
pprint.pprint(trackerdict[trackerkeys[5]])

# If we are showing the result for the single hour in better detail
pprint.pprint(trackerdict[trackerkeys[5]]['AnalysisObj'].__dict__)

# If we are looping for all the hours

for key in trackerdict.keys():
    demo.makeOct1axis(singleindex=key)
    results=demo.analysis1axis(singleindex=key)

print('Accumulated hourly bifi gain for the day: {:0.3}'.format(sum(demo.Wm2Back) / sum(demo.Wm2Front)))